# Install and Imports

In [0]:
import pandas as pd
import string
import numpy as np 

# Mount Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Pre-processing

## NYTimes

In [0]:
blah = pd.read_csv('/content/drive/Shared drives/CIS550_Project/finalCSV/NYTimesBestSellers.csv')
blah = blah.dropna()

In [0]:
blah.columns

Index(['Unnamed: 0', 'rank', 'rank_last_week', 'weeks_on_list', 'isbn',
       'publisher', 'description', 'title', 'author', 'book_image',
       'book_image_width', 'book_image_height', 'amazon_product_url'],
      dtype='object')

In [0]:
nytimes_entity = blah[['isbn', 'rank','rank_last_week', 'weeks_on_list']]

In [0]:
nytimes_entity.to_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/final/csv/NYTimesBestSellers.csv')
# /content/drive/Shared drives/CIS550_Project/finalCSV/NYTimesBestSellers.csv

## Category

Add NYTimes Category with Book Depository

In [0]:
category_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/finalCSV/Category.csv')
book_category_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/finalCSV/BookCategory.csv')

book_category_df

,isbn,categoryId
0,393325792,220.0
1,393325792,233.0
2,393325792,237.0
3,393325792,2644.0
4,393325792,2679.0
...,...,...
4080416,9781444179637,1378.0
4080417,9781444179637,1384.0
4080418,143293600X,1312.0
4080419,143293600X,2578.0


In [0]:
nytimes_category = pd.read_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/updated/nytimesbookcategory.csv')

In [0]:
nytimes_category.drop(['Unnamed: 0'],axis=1, inplace=True)

In [0]:
len(category_df) + len(nytimes_category)

4271

In [0]:
# Common Categories

pd.merge(category_df, nytimes_category, left_on='categoryName', right_on='category', how='inner')['category'].unique()

array(['Architecture', 'Art', 'Crafts & Hobbies', 'Education',
       'Family & Relationships', 'Learning', 'Mathematics', 'Medical',
       'Music', 'Pets', 'Philosophy', 'Photography', 'Psychology',
       'Science'], dtype=object)

In [0]:
pd.merge(category_df, nytimes_category, left_on='categoryName', right_on='category', how='inner')

,categoryId,categoryName,isbn,category
0,107,Architecture,0767919386,Architecture
1,107,Architecture,0767919394,Architecture
2,107,Architecture,0312569378,Architecture
3,107,Architecture,0789327414,Architecture
4,2509,Art,1627792287,Art
...,...,...,...,...
75,2534,Science,0553380168,Science
76,2534,Science,0062464345,Science
77,2534,Science,0307589382,Science
78,2534,Science,0393254607,Science


In [0]:
categories = ['Architecture', 'Art', 'Crafts & Hobbies', 'Education',
       'Family & Relationships', 'Learning', 'Mathematics', 'Medical',
       'Music', 'Pets', 'Philosophy', 'Photography', 'Psychology',
       'Science']

categories_dict = {}

for category in categories:
  cid = category_df[category_df['categoryName'] == category].categoryId.values[0]

  categories_dict[category] = cid

In [0]:
categories_dict

{'Architecture': 107,
 'Art': 2509,
 'Crafts & Hobbies': 2942,
 'Education': 2553,
 'Family & Relationships': 2791,
 'Learning': 771,
 'Mathematics': 1477,
 'Medical': 1279,
 'Music': 165,
 'Pets': 2524,
 'Philosophy': 2722,
 'Photography': 81,
 'Psychology': 750,
 'Science': 2534}

In [0]:
category_df['categoryId'].max()

3390

In [0]:
nytimes_category['categoryId'] = None

In [0]:
# categories = ['Architecture', 'Art', 'Crafts & Hobbies', 'Education',
#        'Family & Relationships', 'Learning', 'Mathematics', 'Medical',
#        'Music', 'Pets', 'Philosophy', 'Photography', 'Psychology',
#        'Science']

counter = 3391
item = []
for i, row in nytimes_category.iterrows():
  cid = categories_dict.get(row.category)
  if cid:
    row.categoryId = cid
  else:
    row.categoryId = counter
    counter += 1
  print(i)  
  item.append(row)   

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [0]:
# nytimes_category

df = pd.DataFrame(item)

df

,isbn,category,categoryId
0,1451627289,Fiction,3391
1,039915731X,Fiction,3392
2,0425232204,Fiction,3393
3,0385535139,Fiction,3394
4,0307272117,Fiction,3395
...,...,...,...
1547,0374298637,Biography & Autobiography,4865
1548,1426215614,Photography,81
1549,1426215649,Travel,4866
1550,0399584692,Biography & Autobiography,4867


In [0]:
nytimes_category

,isbn,category,categoryId
0,1451627289,Fiction,3391
1,039915731X,Fiction,3392
2,0425232204,Fiction,3393
3,0385535139,Fiction,3394
4,0307272117,Fiction,3395
...,...,...,...
1547,0374298637,Biography & Autobiography,4865
1548,1426215614,Photography,81
1549,1426215649,Travel,4866
1550,0399584692,Biography & Autobiography,4867


In [0]:
nytimes_categories = nytimes_category[['categoryId', 'category']]

In [0]:
# nytimes_categories.rename(columns={'category': 'categoryName'}, inplace = True)
nytimes_categories

In [0]:
categories_df = pd.concat([category_df, nytimes_categories])

In [0]:
categories_df = categories_df.drop_duplicates()

In [0]:
categories_df

,categoryId,categoryName
0,1998,.Net Programming
1,176,20th Century & Contemporary Classical Music
2,3291,20th Century & Contemporary Classical Music
3,2659,20th Century History: C 1900 To C 2000
4,2661,21st Century History: From C 2000 -
...,...,...
1546,4864,Biography & Autobiography
1547,4865,Biography & Autobiography
1549,4866,Travel
1550,4867,Biography & Autobiography


In [0]:
categories_df.to_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/final/csv/Category.csv')

## BookCategory

Add NYtimes

In [0]:
book_category_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/finalCSV/BookCategory.csv')

book_category_df

In [0]:
nytimes_bookcategories = nytimes_category[['isbn', 'categoryId']]

nytimes_bookcategories

,isbn,categoryId
0,1451627289,3391
1,039915731X,3392
2,0425232204,3393
3,0385535139,3394
4,0307272117,3395
...,...,...
1547,0374298637,4865
1548,1426215614,81
1549,1426215649,4866
1550,0399584692,4867


In [0]:
# bookcategories_df = pd.concat([book_category_df, nytimes_bookcategories])

# bookcategories_df = bookcategories_df.drop_duplicates()

bookcategories_df

,isbn,categoryId
0,393325792,220
1,393325792,233
2,393325792,237
3,393325792,2644
4,393325792,2679
...,...,...
1547,0374298637,4865
1548,1426215614,81
1549,1426215649,4866
1550,0399584692,4867


In [0]:
bookcategories_df.to_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/final/csv/BookCategory.csv')

## Author

Adding NYTimes

In [0]:
# author_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/final/csv/Author.csv')



authorId       int64
authorName    object
dtype: object

In [0]:
author_df['authorName'] = author_df['authorName'].apply(lambda x: str(x))

In [0]:
author_df

,Unnamed: 0,authorId,authorName
0,0,4872,nan
1,1,172310,# House Press
2,2,59149,#1 Blank Comic Books
3,3,100888,#Value!
4,4,28924,#vettech Life
...,...,...,...
245213,1606,245214,Raymond Queneau
245214,1607,245215,Tomas Ross
245215,1608,245216,Stefan Sremac
245216,1609,245217,Nikos Kazantzakis


In [0]:
# nytimes_authordf = pd.read_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/updated/nytimesbookauthor.csv')
# nytimes_authordf.rename(columns={'Unnamed: 0': 'authorId'}, inplace = True)
nytimes_authordf

,authorId,isbn,author
0,0,1451627289,Stephen King
1,1,039915731X,Tom Clancy
2,2,0425232204,Kathryn Stockett
3,3,0385535139,John Grisham
4,4,0307272117,Stieg Larsson
...,...,...,...
1861,1549,1426215649,National Geographic Society (U.S.)
1862,1549,1426215649,National Geographic
1863,1550,0399584692,Saroo Brierley
1864,1551,0520285956,Rebecca Solnit


In [0]:
common = pd.merge(author_df, nytimes_authordf, left_on='authorName', right_on='author', how='inner')[['authorId', 'authorName']]

common.set_index('authorName', inplace= True)
author_dict = common.to_dict().get('authorId')

In [0]:
max_authorId = author_df['authorId'].max()

max_authorId += 1

In [0]:
# nytimes_authordf['authorId'] = nytimes_authordf['authorId'].apply(lambda x: x + max_authorId)

counter = max_authorId  
item = []
for i, row in nytimes_authordf.iterrows():
  aid = author_dict.get(row.author)
  if aid:
    row.authorId = aid
  else:
    row.authorId = counter
    counter += 1
  print(i)
  item.append(row)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [0]:
nytimes_authordf = pd.DataFrame(item)

In [0]:
nytimes_authors = nytimes_authordf[['authorId', 'author']]

In [0]:
nytimes_authors.rename(columns={'author': 'authorName'}, inplace = True)

nytimes_authors

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,authorId,authorName
0,134302,Stephen King
1,121908,Tom Clancy
2,243700,Kathryn Stockett
3,139042,John Grisham
4,152408,Stieg Larsson
...,...,...
1861,244375,National Geographic Society (U.S.)
1862,399,National Geographic
1863,152315,Saroo Brierley
1864,24298,Rebecca Solnit


In [0]:
authors_df = pd.concat([author_df, nytimes_authors])

In [0]:
authors_df

,authorId,authorName
0,4872,nan
1,172310,# House Press
2,59149,#1 Blank Comic Books
3,100888,#Value!
4,28924,#vettech Life
...,...,...
1861,244375,National Geographic Society (U.S.)
1862,399,National Geographic
1863,152315,Saroo Brierley
1864,24298,Rebecca Solnit


In [0]:
bookMovieAuthors = bookmovie_df[['Author']]

In [0]:
# bookMovieAuthors.reset_index(inplace=True)
bookMovieAuthors.rename(columns={'index':'authorId','Author': 'authorName'}, inplace = True)

bookMovieAuthors

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,authorId,authorName
0,0,Agatha Christie
1,1,Louise L. Lambrichs
2,2,Harry Mulisch
3,3,Nick Hornby
4,4,Louis Begley
...,...,...
1606,1606,Raymond Queneau
1607,1607,Tomas Ross
1608,1608,Stefan Sremac
1609,1609,Nikos Kazantzakis


In [0]:
common = pd.merge(authors_df, bookMovieAuthors, left_on='authorName', right_on='Author', how='inner')[['authorId', 'authorName']]
# df1.set_index('isbn', inplace = True)
common.set_index('authorName', inplace= True)
author_dict = common.to_dict().get('authorId')

# author_dict.get('Maria Stoian')

In [0]:
max_authorId = authors_df['authorId'].max()
max_authorId += 1

max_authorId

244376

In [0]:
# nytimes_authordf['authorId'] = nytimes_authordf['authorId'].apply(lambda x: x + max_authorId)

counter = max_authorId  
item = []
for i, row in bookMovieAuthors.iterrows():
  aid = author_dict.get(row.authorName)
  if aid:
    row.authorId = aid
  else:
    row.authorId = counter
    counter += 1
  print(i)
  item.append(row)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [0]:
bookMovieAuthors = pd.DataFrame(item)

,authorId,authorName
0,39498,Agatha Christie
1,244376,Louise L. Lambrichs
2,244377,Harry Mulisch
3,3580,Nick Hornby
4,289,Louis Begley
...,...,...
1606,245214,Raymond Queneau
1607,245215,Tomas Ross
1608,245216,Stefan Sremac
1609,245217,Nikos Kazantzakis


In [0]:
authors_df = pd.concat([authors_df, bookMovieAuthors])

In [0]:
authors_df = authors_df.drop_duplicates()

In [0]:
authors_df

,authorId,authorName
0,4872,nan
1,172310,# House Press
2,59149,#1 Blank Comic Books
3,100888,#Value!
4,28924,#vettech Life
...,...,...
1606,245214,Raymond Queneau
1607,245215,Tomas Ross
1608,245216,Stefan Sremac
1609,245217,Nikos Kazantzakis


In [0]:
author_df.to_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/final/csv/Author1.csv')

### Adding author Wiki Url

In [0]:
author_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/final/csv/Author1.csv')
authorwiki_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/finalCSV/authorInfo.csv')

In [0]:
# authorwiki_df

df1 = author_df.copy()
df2 = authorwiki_df.copy()


df1['authorName'] = df1['authorName'].apply(lambda x: str(x).replace(' ', '').lower())

df2['authorName'] = df2['authorName'].apply(lambda x: str(x).replace(' ', '').lower())

In [0]:
df3 =df pd.merge(df1, df2, left_on='authorName', right_on='authorName', how='inner')

In [0]:
df3

,Unnamed: 0,Unnamed: 0.1,authorId,authorName,wikiURL
0,475,475,33584,a.a.milne,https://en.wikipedia.org/wiki/A.A._Milne
1,531,531,237362,a.bertramchandler,https://en.wikipedia.org/wiki/A._Bertram_Chandler
2,655,655,151173,a.e.vanvogt,https://en.wikipedia.org/wiki/A._E._van_Vogt
3,773,773,8265,a.j.cronin,https://en.wikipedia.org/wiki/A._J._Cronin
4,1374,1374,149025,a.j.cronin,https://en.wikipedia.org/wiki/A._J._Cronin
...,...,...,...,...,...
1084,245158,1514,245159,michaelmorpurgo,https://en.wikipedia.org/wiki/Michael_Morpurgo
1085,245162,1521,245163,richardadams,https://en.wikipedia.org/wiki/Richard_Adams
1086,245183,1563,245184,flanneryo'connor,https://en.wikipedia.org/wiki/Flannery_O%27Connor
1087,245192,1581,245193,l.frankbaum,https://en.wikipedia.org/wiki/L._Frank_Baum


In [0]:
bookfinalwiki_df = df3[['authorId', 'wikiURL']]

In [0]:
bookfinalwiki_df

,authorId,wikiURL
0,33584,https://en.wikipedia.org/wiki/A.A._Milne
1,237362,https://en.wikipedia.org/wiki/A._Bertram_Chandler
2,151173,https://en.wikipedia.org/wiki/A._E._van_Vogt
3,8265,https://en.wikipedia.org/wiki/A._J._Cronin
4,149025,https://en.wikipedia.org/wiki/A._J._Cronin
...,...,...
1084,245159,https://en.wikipedia.org/wiki/Michael_Morpurgo
1085,245163,https://en.wikipedia.org/wiki/Richard_Adams
1086,245184,https://en.wikipedia.org/wiki/Flannery_O%27Connor
1087,245193,https://en.wikipedia.org/wiki/L._Frank_Baum


In [0]:
bookfinalwiki_df.to_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/final/csv/AuthorWiki.csv')

## BookAuthor

Adding NYTimes

In [0]:
bookAuthor = pd.read_csv('/content/drive/Shared drives/CIS550_Project/finalCSV/BookAuthor.csv')

bookAuthor

,isbn,authorId
0,393325792,1.0
1,184454737X,2.0
2,184454737X,3.0
3,199669171,4.0
4,1444185497,5.0
...,...,...
1566851,1868885976,243697.0
1566852,039332625X,243698.0
1566853,9781444179637,243699.0
1566854,143293600X,15450.0


In [0]:
nytimes_bookauthordf = nytimes_authordf[['isbn', 'authorId']]

In [0]:
bookAuthors = pd.concat([bookAuthor, nytimes_bookauthordf])

In [0]:
# bookAuthors = bookAuthors.drop_duplicates()
bookAuthors

In [0]:
bookAuthors.to_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/final/csv/BookAuthor.csv')

## BookMovie

In [0]:
bookmovie_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/finalCSV/bookMovie.csv')

bookmovie_df

,BookTitle,YearOfPublication,Author,MovieTitle,MovieReleaseYear
0,The A.B.C. Murders,1936,Agatha Christie,The Alphabet Murders,1966
1,À ton image,1998,Louise L. Lambrichs,À ton image,2004
2,De Aanslag,NaN,Harry Mulisch,The Assault,1986
3,About a Boy,1998,Nick Hornby,About a Boy,2002
4,About Schmidt,1996,Louis Begley,About Schmidt,2002
...,...,...,...,...,...
1606,Zazie in the Metro,NaN,Raymond Queneau,Zazie in the Metro,Zazie dans le Métro
1607,De Zesde Mei,2003,Tomas Ross,06/05,2003
1608,Zona Zamfirova,1906,Stefan Sremac,Zona Zamfirova,2002
1609,Zorba the Greek,1946,Nikos Kazantzakis,Zorba the Greek,1964


In [0]:
book_finalmoviedf = bookmovie_df[['BookTitle', 'MovieTitle', 'MovieReleaseYear']]

In [0]:
book_finalmoviedf.to_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/final/csv/bookMovie.csv')

## Books

In [0]:
nytimes_booksdf = pd.read_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/updated/nytimes.csv')
book1_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/updated/working/book1.csv')
book2_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/updated/working/book2.csv')

In [0]:
book2_df.isna().sum()

Unnamed: 0              0
isbn10                  0
title                   0
description             0
url                     0
publisher               0
publication-date        0
rating-avg              0
lang                    0
place_name            183
format_name             2
num_pages           93596
img_url             93523
dtype: int64

In [0]:
# Processing the NYTimes 

nytimes_booksdf.drop(['Unnamed: 0'],axis=1, inplace=True)
nytimes_booksdf.rename(columns={'amazon_product_url': 'url', 'book_image': 'img_url'}, inplace = True)

In [0]:
book1_df.columns

Index(['isbn', 'title', 'description', 'url', 'publisher', 'publication_date',
       'rating-avg', 'lang', 'place_name', 'format_name', 'num_pages',
       'img_url'],
      dtype='object')

In [0]:
# Processing Book Depository part 1

book1_df.drop(['Unnamed: 0'],axis=1, inplace=True)
book1_df.rename(columns={'isbn10': 'isbn', 'publication-date': 'publication_date'}, inplace = True)

In [0]:
book2_df.columns

Index(['isbn', 'title', 'description', 'url', 'publisher', 'publication_date',
       'rating-avg', 'lang', 'place_name', 'format_name', 'num_pages',
       'img_url'],
      dtype='object')

In [0]:
# Processing Book Depository part 2

book2_df.drop(['Unnamed: 0'],axis=1, inplace=True)
book2_df.rename(columns={'isbn10': 'isbn', 'publication-date': 'publication_date'}, inplace = True)

In [0]:
books_df = pd.concat([nytimes_booksdf, book1_df])
books_df = pd.concat([books_df, book2_df])

In [0]:
len(book1_df) + len(book2_df) + len(nytimes_booksdf)

184734

In [0]:
books_df

In [0]:
books_df.to_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/final/csv/Books.csv')

In [0]:
books_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/final/csv/Books1.csv')

In [0]:
books_df

,Unnamed: 0,Unnamed: 0.1,isbn,title,description,publisher,url,img_url,publication_date,num_pages,lang,rating-avg,place_name,format_name
0,0,0,1451627289,11/22/63,An English teacher travels back to 1958 by way...,Scribner,http://www.amazon.com/11-22-63-A-Novel/dp/1451...,https://s1.nyt.com/du/books/images/97814516272...,2011-11-08,864.0,en,NaN,NaN,NaN
1,1,1,039915731X,LOCKED ON,Jack Ryan Jr. must stop an emerging threat fro...,Penguin Group,http://www.amazon.com/Locked-Jack-Ryan-Tom-Cla...,https://s1.nyt.com/du/books/images/97803991573...,2011,853.0,en,NaN,NaN,NaN
2,2,2,0425232204,THE HELP,A young white woman and two black maids in 196...,Penguin Group,http://www.amazon.com/The-Help-Kathryn-Stocket...,https://s1.nyt.com/du/books/images/97814406976...,2009,534.0,en,NaN,NaN,NaN
3,3,3,0385535139,THE LITIGATORS,Partners in a small law firm take on a big cas...,Knopf Doubleday Publishing,http://www.amazon.com/The-Litigators-John-Gris...,https://s1.nyt.com/du/books/images/97803855351...,2011,385.0,en,NaN,NaN,NaN
4,4,4,0307272117,THE GIRL WITH THE DRAGON TATTOO,A hacker and a journalist investigate the disa...,Knopf Doubleday Publishing,http://www.amazon.com/Girl-Dragon-Tattoo-Stieg...,https://s1.nyt.com/du/books/images/97803079494...,2008-09-16,480.0,en,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184651,184729,138986,1590176219,Proper Doctoring,"""People come to us for help. They come for hea...",NYRB Classics,/Proper-Doctoring-David-Mendel/9781590176214,NaN,2013-09-24,NaN,en,3.88,"New York, United States",Paperback
184652,184730,138987,415195624,Using Groups to Help People,This new edition of Using Groups to Help Peopl...,ROUTLEDGE,/Using-Groups-Help-People-Dorothy-Stock-Whitak...,NaN,2001-01-31,NaN,en,4.50,"London, United Kingdom",Paperback
184653,184731,138988,367437481,Dangerous Decade : Taiwan's Security and Crisi...,Taiwan's position looks increasingly precariou...,ROUTLEDGE,/Dangerous-Decade-Brendan-Taylor/9780367437480,NaN,2019-09-18,NaN,en,4.66,"London, United Kingdom",Paperback
184654,184732,138989,1781257736,Shapeshifters : A Doctor's Notes on Medicine &...,Unreliable bodies and shifting symptoms are al...,Wellcome Collection,/Shapeshifters-Gavin-Francis/9781781257739,http://books.google.com/books/content?id=0UOWt...,2018-05-03,282.0,en,3.93,"London, United Kingdom",Hardback


In [0]:
a = books_df['isbn'].values

In [0]:
b = books_df['isbn'].apply(lambda x: len(x))

In [0]:
# books_df[b != 10]

# dup_books_df = books_df[books_df.duplicated(['isbn'])]

# df = books_df.drop_duplicates(['isbn'])

df

,Unnamed: 0,Unnamed: 0.1,isbn,title,description,publisher,url,img_url,publication_date,num_pages,lang,rating-avg,place_name,format_name
0,0,0,1451627289,11/22/63,An English teacher travels back to 1958 by way...,Scribner,http://www.amazon.com/11-22-63-A-Novel/dp/1451...,https://s1.nyt.com/du/books/images/97814516272...,2011-11-08,864.0,en,NaN,NaN,NaN
1,1,1,039915731X,LOCKED ON,Jack Ryan Jr. must stop an emerging threat fro...,Penguin Group,http://www.amazon.com/Locked-Jack-Ryan-Tom-Cla...,https://s1.nyt.com/du/books/images/97803991573...,2011,853.0,en,NaN,NaN,NaN
2,2,2,0425232204,THE HELP,A young white woman and two black maids in 196...,Penguin Group,http://www.amazon.com/The-Help-Kathryn-Stocket...,https://s1.nyt.com/du/books/images/97814406976...,2009,534.0,en,NaN,NaN,NaN
3,3,3,0385535139,THE LITIGATORS,Partners in a small law firm take on a big cas...,Knopf Doubleday Publishing,http://www.amazon.com/The-Litigators-John-Gris...,https://s1.nyt.com/du/books/images/97803855351...,2011,385.0,en,NaN,NaN,NaN
4,4,4,0307272117,THE GIRL WITH THE DRAGON TATTOO,A hacker and a journalist investigate the disa...,Knopf Doubleday Publishing,http://www.amazon.com/Girl-Dragon-Tattoo-Stieg...,https://s1.nyt.com/du/books/images/97803079494...,2008-09-16,480.0,en,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184651,184729,138986,1590176219,Proper Doctoring,"""People come to us for help. They come for hea...",NYRB Classics,/Proper-Doctoring-David-Mendel/9781590176214,NaN,2013-09-24,NaN,en,3.88,"New York, United States",Paperback
184652,184730,138987,415195624,Using Groups to Help People,This new edition of Using Groups to Help Peopl...,ROUTLEDGE,/Using-Groups-Help-People-Dorothy-Stock-Whitak...,NaN,2001-01-31,NaN,en,4.50,"London, United Kingdom",Paperback
184653,184731,138988,367437481,Dangerous Decade : Taiwan's Security and Crisi...,Taiwan's position looks increasingly precariou...,ROUTLEDGE,/Dangerous-Decade-Brendan-Taylor/9780367437480,NaN,2019-09-18,NaN,en,4.66,"London, United Kingdom",Paperback
184654,184732,138989,1781257736,Shapeshifters : A Doctor's Notes on Medicine &...,Unreliable bodies and shifting symptoms are al...,Wellcome Collection,/Shapeshifters-Gavin-Francis/9781781257739,http://books.google.com/books/content?id=0UOWt...,2018-05-03,282.0,en,3.93,"London, United Kingdom",Hardback


In [0]:
len(books_df) - len(df)

1111

In [0]:
dup_books_df

,Unnamed: 0,Unnamed: 0.1,isbn,title,description,publisher,url,img_url,publication_date,num_pages,lang,rating-avg,place_name,format_name
50,54,54,0345803485,FIFTY SHADES OF GREY,An inexperienced college student falls in love...,Vintage,http://www.amazon.com/Fifty-Shades-Grey-Book-T...,https://s1.nyt.com/du/books/images/97803458034...,2012,514.0,en,NaN,NaN,NaN
66,71,71,0316055433,THE GOLDFINCH,After his mother is killed in a museum explosi...,"Little, Brown",http://www.amazon.com/The-Goldfinch-Novel-Puli...,https://s1.nyt.com/du/books/images/97803162486...,NaN,NaN,NaN,NaN,NaN,NaN
72,77,77,0553418025,THE MARTIAN,"Abandoned by his crew, an astronaut embarks on...",Crown,http://www.amazon.com/The-Martian-Novel-Andy-W...,https://s1.nyt.com/du/books/images/97808041390...,2014,369.0,en,NaN,NaN,NaN
109,114,114,1476746583,ALL THE LIGHT WE CANNOT SEE,The lives of a blind French girl and a gadget-...,Scribner,http://www.amazon.com/All-Light-We-Cannot-See/...,https://s1.nyt.com/du/books/images/97814767465...,2014-05-06,531.0,en,NaN,NaN,NaN
115,120,120,144947425X,MILK AND HONEY,Poetic approaches to surviving adversity and l...,Andrews McMeel,http://www.amazon.com/Milk-Honey-Rupi-Kaur/dp/...,https://s1.nyt.com/du/books/images/97814494742...,2015,204.0,en,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180315,180393,134650,006233901X,The World of PostSecret,New York Times Bestseller An addictive collec...,WILLIAM MORROW,/World-PostSecret-Frank-Warren/9780062339010,http://books.google.com/books/content?id=vQkyn...,2014-11-05,288.0,en,4.38,"New York, United States",Hardback
181321,181399,135656,1592407331,Daring Greatly : How the Courage to be Vulnera...,The #1 New York Times bestseller. 1 million co...,GOTHAM BOOKS,/Daring-Greatly-Brene-Brown/9781592407330,http://books.google.com/books/content?id=K4naC...,2012-09-13,287.0,en,4.27,"New York, United States",Hardback
181422,181500,135757,1594481717,A Whole New Mind : Why Right-brainers Will Rul...,New York Times Bestseller,"Riverhead Books,U.S.",/Whole-New-Mind-Daniel-H-Pink/9781594481710,http://books.google.com/books/content?id=3Xb_M...,2008-07-09,275.0,en,3.94,"New York, United States",Paperback
181469,181547,135804,1594484805,Drive : The Surprising Truth About What Motiva...,The New York Times bestseller that gives reade...,"Riverhead Books,U.S.",/Drive-Daniel-H-Pink/9781594484803,http://books.google.com/books/content?id=pYc-D...,2011-05-26,260.0,en,3.94,"New York, United States",Paperback


In [0]:
duplicateRowsDF = dfObj[dfObj.duplicated(['Age', 'City'])]

In [0]:
books_df = books_df[books_df['isbn'] != 'None']

In [0]:
books_df.to_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/final/csv/Booksfinal.csv')

In [0]:
# books_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/Books-temp/final/csv/Books2.csv');

books_df.dropna()

,isbn,title,description,publisher,url,img_url,publication_date,num_pages,lang,rating-avg,place_name,format_name,for-ages
2050,1510705937,Hacks for Minecrafters: Mods : The Unofficial ...,From New York Times bestselling author Megan M...,Sky Pony Press,/Hacks-for-Minecrafters-Mods-Megan-Miller/9781...,http://books.google.com/books/content?id=G6Yds...,2016-03-01,128.0,en,4.57,"London, United Kingdom",Spiral,9-12
2053,9780307020970,LGB Tootle,"In this classic Little Golden Book from 1945, ...","Golden Books Publishing Company, Inc.",/LGB-Tootle-Gertrude-Crampton/9780307020970,http://books.google.com/books/content?id=0Ud40...,2003-03-01,24.0,en,4.16,"London, United Kingdom",Spiral,12-17
2056,9781444003345,The Complete Philosophy Files,"Is there a God, should I eat meat, where does ...",Orion Children's Books,/Complete-Philosophy-Files-Stephen-Law/9781444...,http://books.google.com/books/content?id=lrGpc...,2011-09-01,398.0,en,4.08,"North Adams, United States",Paperback,9+
2060,9781416950066,Identical,"""Perfect on the outside, but behind the Normal...",Margaret K McElderry Books,/Identical-Ellen-Hopkins/9781416950066,http://books.google.com/books/content?id=RVlyD...,2010-12-21,592.0,en,4.31,"London, United Kingdom",Paperback,12-17
2081,9780375832208,Magic Tree House Fact Tracker #14 Ancient Rome...,The #1 bestselling chapter book series of all ...,Random House Inc,/Magic-Tree-House-Fact-Tracker-14-Ancient-Rome...,http://books.google.com/books/content?id=96gbX...,2006-04-25,119.0,en,4.16,"London, United Kingdom",Paperback,9-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
181820,1566868084,The Legend of Zelda : Ocarina of Time Official...,BradyGAMES The Legend of Zelda:Ocarina of Time...,DORLING KINDERSLEY,/Legend-Zelda-BradyGames/9781566868082,http://books.google.com/books/content?id=M5zCA...,1998-11-23,143.0,en,4.22,"New York, United States",Paperback,12-17
181885,1566869021,Nintendo Game Boy Official Games Guide,BradyGAMES Game Boy Secret Codes: This guide ...,DORLING KINDERSLEY,/Nintendo-Game-Boy-Official-Games-Guide-BradyG...,http://books.google.com/books/content?id=JAkk3...,2000-02-01,136.0,en,3.40,"New York, United States",Paperback,12-17
181945,8467861010,El asesinato de la profesora de ciencias,Después de que un grupo de alumnos vuele por l...,Anaya Infantil y Juvenil,/El-asesinato-de-la-profesora-de-ciencias-Jord...,http://books.google.com/books/content?id=U0a0o...,2014-03-01,166.0,es,3.43,Spain,Paperback,9-12
182654,1599612879,"Thor, Son of Asgard : The Heat of Hakurei","In the second book of a saga of courage, adver...",Spotlight,/Thor-Son-Asgard-Akira-Yoshida/9781599612874,http://books.google.com/books/content?id=0qHSe...,2007-01-01,24.0,en,3.61,"Edina, United States",Hardback,9-12


In [0]:
book_depository_dataset_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/book-depository-dataset/dataset.csv')  

In [0]:
books_df = pd.merge(books_df, book_depository_dataset_df[['isbn10', 'for-ages']], left_on='isbn', right_on='isbn10', how='left')

In [0]:
books_df.drop(['Unnamed: 0', 'Unnamed: 0.1','Unnamed: 0.1.1', 'isbn10' ],axis=1, inplace=True)

In [0]:
book_depository_dataset_df = pd.read_csv('/content/drive/Shared drives/CIS550_Project/book-depository-dataset/dataset.csv')  